# Bayesian optimization of a time-consuming simulator

The aim of the lab is to optimize a time-consuming simulator using the Efficient Global Optimization (EGO) method. As a toy example, the simulator chosen here mimics a catapult. There are 4 input variables, tuning the catapult, and 1 output, giving the distance of the projectile mark to the catapult. We want to find the value(s) of the input variable(s) maximizing that distance. Notice that the simulator is not time-consuming. However, for the sake of realism, we will limit the computational budget to 36 runs.

## "By-hand" Optimization
<br> We provide here a shiny application (authored by Nicolas Durrande), which allows using the simulator interactively. Alternatively, you can use the web application, available here : https://durrande.shinyapps.io/catapult/

In [ ]:
library(shiny)
# runApp()

*Question: By playing with the simulator, propose a set of input values giving the largest possible value of the output. Here we consider the noise-free case, by fixing the wind to zero.*

To continue running the notebook, you may need to interrupt the kernel (square symbol!). Then load the two following scripts, containing useful functions. 

In [ ]:
source("catapultSettings.R")
source("catapultFunctions.R")

## Design of experiments
Let us create an initial design of experiments and compute the corresponding values.

In [ ]:
library(DiceDesign)
set.seed(0)
ninit <- 16
d <- 4
X0 <- lhsDesign(n = ninit, dimension = d)$design
Xopt <- maximinESE_LHS(X0, it = 10)
## you may be interested in the convergence
#plot(Xopt$critValues,type="l")
X <- Xopt$design
colnames(X) <- c("axis", "arm", "spring1", "spring2")
pairs(X)
## compute the output values
Y <- apply(X, 1, runExperiment, windFactor = 0)[1, ]

Question : Observe that the design of experiments is "space-filling". <br> Why did we chose that kind of designs? What is the current maximum? Is it far from the maximum value found by-hand?

## Descriptive statistics
*Question : Can you see a simple input-output relation? What can you say about the area corresponding to the maximum value?*

In [ ]:
pairs(cbind(Y, X))

## Regression metamodel
<br> *Question : Try the linear models below. If you replace the simulator by one of this model, what would be the optimum? Is it far from your previous guess?*

In [ ]:
myData <- data.frame(X, Y = Y)
mReg <- lm(Y ~ ., data = myData)
summary(mReg)
mReg2 <- lm(Y~ . + I(axis^2) + I (arm^2) + I (spring1^2) + I(spring2^2), data = myData)
summary(mReg2)
mStep <- step(object = mReg, scope = mReg2, direction = "both", k = log(length(Y)))
summary(mStep)

## Bayesian optimization
Now, let us try the EGO method (Bayesian optimization)
<br> *Question : Recall its main principles.*

In [ ]:
library(DiceKriging)
mGP0 <- km(~ 1, design = X, response = Y, multistart = 5)
print(mGP0)   # display model
plot(mGP0)    # visual model validation

*Questions: What is the role of the argument 'multistart'? What can you say about the performance and the validity of this first model?*

In [ ]:
library(DiceOptim)

## We first transform the problem to a minimization one
runExperimentFun <- function(x) {
    - runExperiment(x, wind = 0)[1]
}
Y <- apply(X, 1, runExperimentFun)
mGP0 <- km(~ 1, design = X, response = Y)

## Step 1 ##
oEGO <- max_EI(model = mGP0, lower = rep(0, d), upper = rep(1, d))
newX <- oEGO$par
newy <- runExperimentFun(newX)

cat("Expected improvement was:", round(oEGO$value, 2),
    "\nActual improvement is:", round(min(Y) - newy, 2),
    "\n   (>0 means the new point is better, <0 means it is worse)")

In [ ]:
# Then we update the model
mGP1 <- update(mGP0, newX, newy)
# and maximize again the expected improvement: 
## Step 2 ##
oEGO <- max_EI(model = mGP1, lower = rep(0, d), upper = rep(1, d))
newX <- oEGO$par
newy <- runExperimentFun(newX)

cat("Expected improvement was:", round(oEGO$value, 2),
    "\nActual improvement is:", round(min(mGP1@y) - newy, 2),
    "\n   (>0 means the new point is better, <0 means its worst)")

In [ ]:
# Do a loop to automatize the process (for noise-free observations, you can also use EGO.nsteps)
nsteps <- 20
mGP <- mGP0
EGOpar <- matrix(NA, nrow = nsteps, ncol = d)
EGOvalue <- rep(NA, nsteps)
for (i in 1:nsteps){
    oEGO <- max_EI(model = mGP, lower = rep(0, d), upper = rep(1, d))
    newX <- EGOpar[i, ] <- oEGO$par
    newy <- EGOvalue[i] <- runExperimentFun(newX)
    mGP <- update(mGP, newX, newy)
}

bestIndex <- which.min(EGOvalue)
cat("longest shot observed:", - round(EGOvalue[bestIndex], 2),
    "\ncorresponding input values:", round(EGOpar[bestIndex, ], 2))


Global minimum found by EGO:

In [ ]:
EGOpar[bestIndex, ]
runExperimentFun(EGOpar[bestIndex, ])

Let us visualize the 20 points computed with EGO in the X-Y space, and in time order.

In [ ]:
visualizeEGO <- function(initDesign, initValues, EGOpoints, EGOvalues){
  bestIndex <- which.min(EGOvalues)
  y <- c(initValues, EGOvalues, EGOvalues[bestIndex])
  X <- rbind(initDesign, EGOpoints, EGOpoints[bestIndex, ])
  nsteps <- nrow(EGOpoints)
  pairs(cbind(y, X), 
        col = c(rep("black", ninit), rep("blue", nsteps), "red"),
        pch = c(rep(1, ninit), rep(19, nsteps + 1)))
}

visualizeEGO(initDesign = X, initValues = Y,
             EGOpoints = EGOpar, EGOvalues = EGOvalue)

In [ ]:
plot(c(Y, EGOvalue), main = "convergence", xlab = "evaluation number", ylab = "Y values")
lines(rep(ninit, 2), range(Y, EGOvalue), lty = 2, col = "gray")
lines(ninit + 0:nsteps, cummin(c(min(Y), EGOvalue)), col = "red", lwd = 2)

 Questions : 
 * *Why the EGO method would be much less efficient by using a linear model instead of a GP model?*
 * *Modify the code in order to deal with noisy observations, with the EQI criterion (mind using the argument 'noise.var' in 'km').*
 * *Investigate the influence: [a] of a trend in the model (change 'formula' in 'km'); [b] of a kernel; [c] of the initial sample size.*
 * *Adapt the EGO method in order to provide a batch of 2 points at one (function qEGO.nsteps), which is useful in practice, as the 2 runs of the time-consuming simulator can be done in parallel.*

Bonus : As the function is quick to evaluate, compare the result with the maximum value obtained with a standard optimizor (from library nloptr for instance). You can use the 'runExperimentFun' function defined above.